In [1]:
%run config.ipynb
import time
import datetime
import pandas as pd
import numpy as np
import feather
import threading
import queue
from pandas import json_normalize
from binance.spot import Spot
import itertools
from binance.lib.utils import config_logging
from binance.websocket.spot.websocket_stream import SpotWebsocketStreamClient
import logging
import orjson
# import json
# to specify the type of the columns parameter in the __init__ method of the Data class
from typing import List

pd.set_option('display.float_format', lambda x: '%.8f' % x)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [2]:
def adjust_data_to_table(message):
    """
    use this function to get pandas DataFrame from stream from binance
    message : for binance message_handler(_, message)
    pairs_list : list of pairs to get data in stream from binance,
    get this with get_triangular_variants(main_cur, currencies)
    """
    # E: time
    # s: Symbol
    # b: Best bid price
    # B: Best bid quantity
    # a: Best ask price
    # A: Best ask quantity
    # specify symbols from original binance response
    # full response here https://binance-docs.github.io/apidocs/spot/en/#individual-symbol-ticker-streams
    # key_data = ['E', 's', 'b', 'B', 'a', "A"]

    # if 'e' in message:
    # convert json response to dictionary
    json_message = orjson.loads(message)

    # turn to pandas DF and rename to make things comfortable
    triangular_table = pd.DataFrame(json_message)
    
    triangular_table = triangular_table[['s', 'b', 'B', 'a', "A"]].rename(columns={'s':'symbol', 'b':'bidPrice',
                                                   'B':'bidQty', 'a':'askPrice', "A":'askQty'})

    # adjust timestamp and convert values to float
#     triangular_table['Time'] = pd.to_datetime(triangular_table['Time'],unit='ms')

#     triangular_table['bidPrice'] = triangular_table['bidPrice'].astype(float)
#     triangular_table['bidQty'] = triangular_table['bidQty'].astype(float)
#     triangular_table['askPrice'] = triangular_table['askPrice'].astype(float)
#     triangular_table['askQty'] = triangular_table['askQty'].astype(float)
    
    # try this
    triangular_table[['bidPrice','bidQty','askPrice','askQty']] = triangular_table[['bidPrice','bidQty','askPrice','askQty']].astype(float)

        # save table
        # triangular_table.to_csv('practice_table.csv')

        # return full table in DataFrame
    return triangular_table

In [3]:
def get_tatable():
    cl = Spot(api_key=api_key, api_secret=secret_key)
    exchange_info = cl.exchange_info()
    symbols = exchange_info['symbols']

    # per symbol gather base-quote currencies
    quote_dash_pairs = [symbol['baseAsset'] + '-' + symbol['quoteAsset']
                        for symbol in symbols
                        if symbol["status"] == "TRADING" and 'MARKET' in
                        symbol['orderTypes'] and 'SPOT' in
                        symbol['permissions']]

    # remove -(dash) to check dash_quote_pairs vs quote_pairs
    quote_undashed_pairs = []
    for x in quote_dash_pairs:
        quote_undashed_pairs.append(x.split('-')[0] + x.split('-')[1])

    # get 24 hours statistics to know what tickers to use
    y = cl.ticker_24hr()
    all_info = pd.DataFrame(y)
    all_info['openTime'] = pd.to_datetime(all_info['openTime'], unit='ms')
    all_info['closeTime'] = pd.to_datetime(all_info['closeTime'], unit='ms')
    all_info = all_info[['symbol', 'weightedAvgPrice', 'volume', 'quoteVolume', 'openTime', 'closeTime', 'count']]
    all_info = all_info[all_info['symbol'].isin(quote_undashed_pairs)]
    # count is the number for operations for that curr per 24h
    all_info = all_info[all_info['count'] > 250].sort_values('count', ascending=False)
    trading_pairs_undashed = list(all_info['symbol'])
    
    # make liquidity lists, the number - the amount of operations per 24h
    liquid_pairs = all_info[all_info['count'] > 200000]
    liquid_list = list(liquid_pairs['symbol'])
    
    illiquid_pairs = all_info[all_info['count'] < 10000]
    illiquid_list = list(illiquid_pairs['symbol'])
    
    semiliquid = set(trading_pairs_undashed) - set(liquid_list) - set(illiquid_list)
    semiliquid_list = list(semiliquid)
    
    # get all dashed quoted pairs with count parameter
    dashed_filtered_pairs = []
    for quote_pair in quote_dash_pairs:
        dashless_pair = quote_pair.replace('-', '')
        if dashless_pair in list(all_info['symbol']):
            dashed_filtered_pairs.append(quote_pair)

    init_tuple = ('USDT', 'USDC', 'BUSD')
#     for bnb equivalent of 1000 usdt (approx. 3 bnb)
#     init_tuple = ('USDT', 'BNB', 'BUSD')
    
    filtered_pairs_dashed = [pair for pair in dashed_filtered_pairs if pair.endswith(init_tuple)]
    pair_rates_dashed = list(itertools.combinations(filtered_pairs_dashed, 2))

    flat_pairs = [elem for pair in pair_rates_dashed for elem in pair]
    # take every 2 elements and turn them into lists
    pairs = [[flat_pairs[i], flat_pairs[i + 1]] for i in range(0, len(flat_pairs) - 1, 2)]

    copy_pairs = list(pairs)
    # create possible triangular arbitrage variants for all tickers! for itertools combinations
    for pair in copy_pairs:
        if pair[0].split('-')[1] == pair[1].split('-')[1]:
            combo1 = pair[0].split('-')[0]
            combo2 = pair[1].split('-')[0]
            if combo1 + combo2 in trading_pairs_undashed:
                pair.insert(1, combo1 + '-' + combo2)
            elif combo2 + combo1 in trading_pairs_undashed:
                pair.insert(1, combo2 + '-' + combo1)
                # if no matches clear pair
            else:
                pair.clear()

        elif pair[0].split('-')[1] == pair[1].split('-')[0]:
            combo1 = pair[0].split('-')[0]
            combo2 = pair[1].split('-')[1]
            if combo1 + combo2 in trading_pairs_undashed:
                pair.insert(1, combo1 + '-' + combo2)
            elif combo2 + combo1 in trading_pairs_undashed:
                pair.insert(1, combo2 + '-' + combo1)
                # if no matches clear pair
            else:
                pair.clear()

        elif pair[0].split('-')[0] == pair[1].split('-')[0]:
            combo1 = pair[0].split('-')[1]
            combo2 = pair[1].split('-')[1]
            if combo1 + combo2 in trading_pairs_undashed:
                pair.insert(2, combo1 + '-' + combo2)
            elif combo2 + combo1 in trading_pairs_undashed:
                pair.insert(2, combo2 + '-' + combo1)
                # if no matches clear pair
            else:
                pair.clear()
        else:
            pair.clear()
    new_data = list(filter(bool, copy_pairs))

    a_pair_list = [x[0] for x in new_data]
    b_pair_list = [x[1] for x in new_data]
    c_pair_list = [x[2] for x in new_data]
    static_data_table = pd.DataFrame({'a_pair': a_pair_list, 'b_pair': b_pair_list, 'c_pair': c_pair_list})

    static_data_table = static_data_table.drop_duplicates()
    
    # need to remove occurrences of TUSD-T and T-USDT in one row
    mask = ((static_data_table == 'TUSD-T').sum(axis=1) >= 1) & ((static_data_table == 'T-USDT').sum(axis=1) >= 1)
    static_data_table = static_data_table[~mask]

    static_data_table = static_data_table.reset_index(drop=True)
    unique_pairs = set(static_data_table['a_pair']).union(set(static_data_table['b_pair'])).union(set(static_data_table['c_pair']))
    unique_list = [*unique_pairs, ]
    
    # list of unique undashed tickers
    unique_undashed_list = []
    for pair in unique_list:
        unique_undashed_list.append(pair.split('-')[0] + pair.split('-')[1])
    
    # make df to check liquidity fro all tickers in static data table (TA variants)
    # we need it to filter steps of TA, like liquid and semiliquid tickers must be used in 
    # 1 step (a_pair) and 2 step (b_pair), and 3 step must use (illiquid tickers)
    check_liquidity = pd.DataFrame({'dashed_tickers': unique_list, 'undashed_tickers':unique_undashed_list})
    conditions1 = [
        check_liquidity['undashed_tickers'].isin(liquid_list),
        check_liquidity['undashed_tickers'].isin(semiliquid_list),
        check_liquidity['undashed_tickers'].isin(illiquid_list)
    ]

    values = ['liquid', 'semiliquid', 'illiquid']
    
    # make a mirrored version of ta variants and concat it to static datatable
    reversed_static_table = static_data_table.iloc[:, ::-1]
    reversed_static_table.columns = ['a_pair','b_pair','c_pair']
    
    check_liquidity['liquidity'] = np.select(conditions1, values)
        
    full_static_table = pd.concat([static_data_table,reversed_static_table])
    
    # maked dashed liquidity lists
    final_liquidity_check = check_liquidity[['dashed_tickers','liquidity']]
    dashed_liquid_list = final_liquidity_check.loc[final_liquidity_check['liquidity'] == 'liquid', 'dashed_tickers'].tolist()
    dashed_semiliquid_list = final_liquidity_check.loc[final_liquidity_check['liquidity'] == 'semiliquid', 'dashed_tickers'].tolist()
    dashed_illiquid_list = final_liquidity_check.loc[final_liquidity_check['liquidity'] == 'illiquid', 'dashed_tickers'].tolist()    
       
    condition2 = ((full_static_table['a_pair'].isin(dashed_liquid_list+dashed_semiliquid_list)) & 
             (full_static_table['b_pair'].isin(dashed_liquid_list+dashed_semiliquid_list)) & 
             (full_static_table['c_pair'].isin(dashed_semiliquid_list+dashed_illiquid_list)))
    final_static_table = full_static_table[condition2].reset_index(drop=True)
#     final_static_table = final_static_table
        
    return final_static_table

In [4]:
def ta_preparation(table_variants, tickers_data_df):
    
    # first part of function - get every pair bid and ask values and transform it into one dataframe
    static1 = table_variants[['a_pair']]
    static1 = static1['a_pair'].str.replace('-','')
    static1 = static1.to_frame()
    merge_static1 = pd.merge(static1,tickers_data_df, left_on='a_pair', right_on='symbol',how='left').drop(columns=['symbol'])
    merge_static1['a_pair'] = table_variants['a_pair']
    merge_static1 = merge_static1.rename(columns={'bidPrice': 'a_pair_bid_p', 
                                                  'bidQty': 'a_pair_ask_q',
                                                  'askPrice': 'a_pair_ask_p',
                                                  'askQty': 'a_pair_ask_q'})
    # merge_static1

    static2 = table_variants[['b_pair']]
    static2 = static2['b_pair'].str.replace('-','')
    static2 = static2.to_frame()
    merge_static2 = pd.merge(static2,tickers_data_df, left_on='b_pair', right_on='symbol',how='left').drop('symbol',axis=1)
    merge_static2['b_pair'] = table_variants['b_pair']
    merge_static2 = merge_static2.rename(columns={'bidPrice': 'b_pair_bid_p', 
                                                  'bidQty': 'b_pair_ask_q',
                                                  'askPrice': 'b_pair_ask_p',
                                                  'askQty': 'b_pair_ask_q'})
    # merge_static2

    static3 = table_variants[['c_pair']]
    static3 = static3['c_pair'].str.replace('-','')
    static3 = static3.to_frame()
    merge_static3 = pd.merge(static3,tickers_data_df, left_on='c_pair', right_on='symbol',how='left').drop(columns=['symbol'])
    merge_static3['c_pair'] = table_variants['c_pair']
    merge_static3 = merge_static3.rename(columns={'bidPrice': 'c_pair_bid_p', 
                                                  'bidQty': 'c_pair_ask_q',
                                                  'askPrice': 'c_pair_ask_p',
                                                  'askQty': 'c_pair_ask_q'})
    # merge_static3

    a_t = pd.concat([merge_static1, merge_static2, merge_static3], axis=1)
#     the problem is that some ticker's value doesn't exist, so there are empty values for pairs
    a_t = a_t.dropna()
    
    # second part - calculate cross-spreads
    
    a_right = a_t['a_pair'].str.split('-').str[1]
    a_left = a_t['a_pair'].str.split('-').str[0]
    c_right = a_t['c_pair'].str.split('-').str[1]
    b_left = a_t['b_pair'].str.split('-').str[0]
    b_right = a_t['b_pair'].str.split('-').str[1]

    conditions = [(a_right == c_right) &
                  (a_left == b_right),

                 (a_right == c_right) &
                  (a_left == b_left),

                 (a_right != c_right) &
                  (a_left == b_right),

                 (a_right != c_right) &
                  (a_left != b_right)
    ]

    a_pair_bid = a_t['a_pair_bid_p']
    a_pair_ask = a_t['a_pair_ask_p']
    c_pair_bid = a_t['c_pair_bid_p']
    c_pair_ask = a_t['c_pair_ask_p']

    bid_calculations = [(1/a_pair_ask)*c_pair_bid,
                        a_pair_bid*(1/c_pair_ask),
                        1/(a_pair_bid/c_pair_bid),
                        a_pair_bid*c_pair_bid
                       ]

    ask_calculations = [(1/a_pair_bid)*c_pair_ask,
                        a_pair_ask*(1/c_pair_bid),
                        1/(a_pair_ask/c_pair_ask),
                        a_pair_ask*c_pair_ask
                       ]

    a_t['calc_bid'] = np.select(conditions, bid_calculations)
    a_t['calc_ask'] = np.select(conditions, ask_calculations)
    
    # third part - make boolean column that shows satisfying cross spreads and filter
    a_t['arb_op'] = np.where((a_t['calc_ask'] < a_t['b_pair_bid_p']) | (a_t['b_pair_ask_p'] < a_t['calc_bid']), True, False)
    a_t_T = a_t[a_t['arb_op'] == True]
    
    return a_t_T

In [5]:
def ta_calculation(a_t_T):
    t_profits = pd.DataFrame(columns = ['Time', 'Descripion', 'Profit'])
    init_money = 1000
#     fee = 0.0002 # vip
    fee = 0.00075 # binance BNB
#     fee = 0.001
    init_currency = ''
    # acquired_coin_s1 = ''
    # acquired_coin_s2 = ''
    # acquired_coin_s3 = ''
    # action1 = ''
    # action2 = ''
    # action3 = ''
    step1_res = 0
    step2_res = 0
    step3_res = 0
    step_decr1 = ''
    step_decr2 = ''
    step_decr3 = ''
    arb_profit = 0
    result_list = []

    for item in a_t_T.itertuples():
        
        a_pair = item[1]
        a_pair_bid_price = item[2]
        a_pair_ask_price = item[4]
        b_pair = item[6]
        b_pair_bid_price = item[7]
        b_pair_ask_price = item[9]
        c_pair = item[11]
        c_pair_bid_price = item[12]
        c_pair_ask_price = item[14]

        # Scenario 1 
        if str(a_pair).split('-')[1] == str(c_pair).split('-')[1] and str(a_pair).split('-')[0] == str(b_pair).split('-')[1]:
            # up the ask and divide buy
            step1_res = (init_money / a_pair_ask_price) * (1-fee)
            step_decr1 = f"1.Start with {init_money} {str(a_pair).split('-')[1]} and buy {step1_res} {str(item[1]).split('-')[0]} "

            # up the ask and divide buy
            step2_res = (step1_res / b_pair_ask_price) * (1-fee)
            step_decr2 = f"2.Buy {step2_res} {str(b_pair).split('-')[0]} with {str(b_pair).split('-')[1]} "

            # down the bid and multiply sell
            step3_res = (step2_res * c_pair_bid_price) * (1-fee)
            step_decr3 = f"3.Sell {str(c_pair).split('-')[0]} to get {step3_res} {str(c_pair).split('-')[1]}. Scenario 1."

            arb_profit = step3_res - init_money
    #         print(step_decr1,step_decr2,step_decr3,f'For a profit of {arb_profit}',sep='\n')

        # Scenario 2
        elif str(a_pair).split('-')[1] == str(c_pair).split('-')[1] and str(a_pair).split('-')[0] == str(b_pair).split('-')[0]:
            # up the ask and divide buy
            step1_res = (init_money / a_pair_ask_price) * (1-fee)
            step_decr1 = f"1.Start with {init_money} {str(a_pair).split('-')[1]} and buy {step1_res} {str(a_pair).split('-')[0]} "

            # down the bid and multiply sell
            step2_res = (step1_res * b_pair_bid_price) * (1-fee)
            step_decr2 = f"2.Sell {str(b_pair).split('-')[0]} to get {step2_res} {str(b_pair).split('-')[1]}  "

            # down the bid and multiply sell
            step3_res = (step2_res * c_pair_bid_price) * (1-fee)
            step_decr3 = f"3.Sell {str(c_pair).split('-')[0]} to get {step3_res} {str(c_pair).split('-')[1]}. Scenario 2."

            arb_profit = step3_res - init_money
    #         print(step_decr1,step_decr2,step_decr3,f'For a profit of {arb_profit}',sep='\n')

        # Scenario 3
        elif str(a_pair).split('-')[1] == str(c_pair).split('-')[0] and str(a_pair).split('-')[0] == str(b_pair).split('-')[0]:
            # up the ask and divide buy
            step1_res = (init_money / a_pair_ask_price) * (1-fee)
            step_decr1 = f"1.Start with {init_money} {str(a_pair).split('-')[1]} and buy {step1_res} {str(a_pair).split('-')[0]} "

            # down the bid and multiply sell
            step2_res = (step1_res * b_pair_bid_price) * (1-fee)
            step_decr2 = f"2.Sell {str(b_pair).split('-')[0]} to get {step2_res} {str(b_pair).split('-')[1]} "

            # up the ask and divide buy
            step3_res = (step2_res / c_pair_ask_price) * (1-fee)
            step_decr3 = f"3.Buy {step3_res} {str(c_pair).split('-')[0]} with {str(c_pair).split('-')[1]}. Scenario 3."

            arb_profit = step3_res - init_money
    #         print(step_decr1,step_decr2,step_decr3,f'For a profit of {arb_profit}',sep='\n')

            # Scenario 4
        elif str(a_pair).split('-')[1] == str(c_pair).split('-')[0] and str(a_pair).split('-')[0] == str(b_pair).split('-')[1]:
            # up the ask and divide buy
            step1_res = (init_money / a_pair_ask_price) * (1-fee)
            step_decr1 = f"1.Start with {init_money} {str(a_pair).split('-')[1]} and buy {step1_res} {str(a_pair).split('-')[0]} "

            # up the ask and divide buy
            step2_res = (step1_res / b_pair_ask_price) * (1-fee)
            step_decr2 = f"2.Buy {str(b_pair).split('-')[0]} with {step2_res} {str(b_pair).split('-')[1]} "

            # up the ask and divide buy
            step3_res = (step2_res / c_pair_ask_price) * (1-fee)
            step_decr3 = f"3.Buy {step3_res} {str(c_pair).split('-')[0]} with {str(c_pair).split('-')[1]}. Scenario 4."

            arb_profit = step3_res - init_money
    #         print(step_decr1,step_decr2,step_decr3,f'For a profit of {arb_profit}',sep='\n')

            # the reversed part could be a subject of recreation depending on final_static_table variants
            # this example has builtin reversed variants, so there no scenario 5 and 6

                # Scenario 7
        elif str(a_pair).split('-')[0] == str(c_pair).split('-')[1] and str(a_pair).split('-')[1] == str(b_pair).split('-')[1]:
            # down the bid and multiply sell
            step1_res = (init_money * a_pair_bid_price) * (1-fee)
            step_decr1 = f"1.Start with {init_money} {str(a_pair).split('-')[0]} and sell for {step1_res} {str(a_pair).split('-')[1]} "

            # up the ask and divide buy
            step2_res = (step1_res / b_pair_ask_price) * (1-fee)
            step_decr2 = f"2.Buy {str(b_pair).split('-')[0]} with {step2_res} {str(b_pair).split('-')[1]} "

            # down the bid and multiply sell
            step3_res = (step2_res * c_pair_bid_price) * (1-fee)
            step_decr3 = f"3.Sell {str(c_pair).split('-')[0]} to get {step3_res} {str(c_pair).split('-')[1]}. Scenario 7."

            arb_profit = step3_res - init_money
    #               print(step_decr1,step_decr2,step_decr3,f'For a profit of {arb_profit}',sep='\n')

                # Scenario 8
        elif str(a_pair).split('-')[0] == str(c_pair).split('-')[1] and str(a_pair).split('-')[1] == str(b_pair).split('-')[0]:
            # down the bid and multiply sell
            step1_res = (init_money * a_pair_bid_price) * (1-fee)
            step_decr1 = f"1.Start with {init_money} {str(a_pair).split('-')[0]} and sell for {step1_res} {str(a_pair).split('-')[1]} "

            # down the bid and multiply sell
            step2_res = (step1_res * b_pair_bid_price) * (1-fee)
            step_decr2 = f"2.Sell {str(b_pair).split('-')[0]} to get {step2_res} {str(b_pair).split('-')[1]} "

            # down the bid and multiply sell
            step3_res = (step2_res * c_pair_bid_price) * (1-fee)
            step_decr3 = f"3.Sell {str(c_pair).split('-')[0]} to get {step3_res} {str(c_pair).split('-')[1]}. Scenario 8."

            arb_profit = step3_res - init_money
        #               print(step_decr1,step_decr2,step_decr3,f'For a profit of {arb_profit}',sep='\n')

        message = step_decr1 +''+step_decr2 +''+step_decr3+''+a_pair+'/'+b_pair+'/'+c_pair
#         message = step_decr1 +''+step_decr2 +''+step_decr3
        calculaton_time = pd.Timestamp(datetime.datetime.now())
        t_profits.loc[len(t_profits)] = [calculaton_time, message, float(arb_profit)]

    
    return t_profits.sort_values(by='Profit',ascending=False).head(1)

In [6]:
# variant with Event threading

class Arbitrage:
    print('Initializing...')
    
    def __init__(self):
        self.ta_table = get_tatable()
        self.initial_df = pd.DataFrame(columns=['symbol', 'bidPrice', 'bidQty', 'askPrice', 'askQty'])
        self.calculations_df = pd.DataFrame(columns=['Time', 'Descripion', 'Profit'])
        self.count = 0
        self.event = threading.Event()
        # set the event initially to True
        self.event.set()
        
        ta_table_thread = threading.Thread(target=self.update_ta_table, daemon=True)
        ta_table_thread.start()

    def update_ta_table(self):
        while self.event.is_set():
            
            self.ta_table = get_tatable()
            print('TA table is updated')
            time.sleep(15)

    def message_handler(self, _, message):
        # remove the problem with Unnamed:0
        x = adjust_data_to_table(message).reset_index(drop=True)

        # check if the the downloading goes every second
        self.count += 1

        # just checking where the process is
        print(f'{self.count} downloaded {len(message)} units of data')

        # creating udpdatable pandas dataframe table
        self.initial_df = pd.concat([self.initial_df, x])

        # remove duplicates with new data
        self.initial_df = self.initial_df.drop_duplicates(subset='symbol', keep='last')

        prepare = ta_preparation(self.ta_table, self.initial_df)
        calculate = ta_calculation(prepare)

        # make dataframe to save calculations to .csv file
        self.calculations_df = pd.concat([self.calculations_df, calculate])
        # self.calculations_df.to_csv('profits.csv', index=False)
            
        # try feather instead of csv
        feather.write_dataframe(self.calculations_df, 'profits.feather')
    
    def trading(self):

        my_client = SpotWebsocketStreamClient(on_message=self.message_handler)
        my_client.ticker()
#          the duration of datastream
        time.sleep(40)

        # unsubscribe
        my_client.ticker(action=SpotWebsocketStreamClient.ACTION_UNSUBSCRIBE)

        # the pause after datastream ended
        time.sleep(10)
        logging.debug("closing ws connection")
#         print("closing ws connection")
        my_client.stop()
        self.event.clear()
        # clear the event to stop the ta_table_thread

bot = Arbitrage()
bot.trading()

Initializing...


Error from callback <bound method Arbitrage.message_handler of <__main__.Arbitrage object at 0x00000234797EA910>>: If using all scalar values, you must pass an index


1 downloaded 135767 units of data
2 downloaded 194234 units of data
3 downloaded 153033 units of data
4 downloaded 132973 units of data
5 downloaded 140714 units of data
6 downloaded 151998 units of data
TA table is updated
7 downloaded 122756 units of data
8 downloaded 114281 units of data
9 downloaded 102989 units of data
10 downloaded 99441 units of data
11 downloaded 112612 units of data
12 downloaded 122469 units of data
13 downloaded 129970 units of data
14 downloaded 167701 units of data
15 downloaded 99706 units of data
16 downloaded 291007 units of data
17 downloaded 254594 units of data
18 downloaded 369968 units of data
19 downloaded 318887 units of data
20 downloaded 376697 units of data
21 downloaded 354572 units of data
22 downloaded 261618 units of data
23 downloaded 215772 units of data
24 downloaded 174296 units of data
25 downloaded 218168 units of data
26 downloaded 251993 units of data
27 downloaded 259324 units of data
28 downloaded 242433 units of data
29 download

Error from callback <bound method Arbitrage.message_handler of <__main__.Arbitrage object at 0x00000234797EA910>>: If using all scalar values, you must pass an index


40 downloaded 192998 units of data


TA table is updated


In [7]:
# dfc = pd.read_csv('profits.csv')
# dfc.sort_values(by='Profit',ascending=False)
dfcf = feather.read_dataframe('profits.feather')
dfcf.sort_values(by='Profit',ascending=False).head(20)

,Time,Descripion,Profit
33,2023-05-10 17:37:04.024990,1.Start with 1000 BUSD and sell for 999.05015 USDT 2.Buy IDEX with 10742.503630555257 USDT 3.Sell IDEX to get 998.3035480134076 BUSD. Scenario 7.BUSD-USDT/IDEX-USDT/IDEX-BUSD,-1.69645199
40,2023-05-10 17:37:01.033492,1.Start with 1000 BUSD and sell for 999.05015 USDT 2.Buy OP with 570.45763565 USDT 3.Sell OP to get 998.1221665331326 BUSD. Scenario 7.BUSD-USDT/OP-USDT/OP-BUSD,-1.87783347
16,2023-05-10 17:37:04.923266,1.Start with 1000 USDT and buy 538.6792452830189 RNDR 2.Sell RNDR to get 999.0388377358491 BUSD 3.Sell BUSD to get 998.0899006958258 USDT. Scenario 2.RNDR-USDT/RNDR-BUSD/BUSD-USDT,-1.91009930
21,2023-05-10 17:37:01.980683,1.Start with 1000 USDT and buy 538.099084544965 RNDR 2.Sell RNDR to get 999.0382580102315 BUSD 3.Sell BUSD to get 998.0893215208605 USDT. Scenario 2.RNDR-USDT/RNDR-BUSD/BUSD-USDT,-1.91067848
20,2023-05-10 17:36:59.930933,1.Start with 1000 BUSD and sell for 999.05015 USDT 2.Buy STG with 1461.8551213757505 USDT 3.Sell STG to get 997.9903643597198 BUSD. Scenario 7.BUSD-USDT/STG-USDT/STG-BUSD,-2.00963564
39,2023-05-10 17:37:03.052595,1.Start with 1000 BUSD and sell for 999.05015 USDT 2.Buy LUNC with 10893723.94573876 USDT 3.Sell LUNC to get 997.9875588868206 BUSD. Scenario 7.BUSD-USDT/LUNC-USDT/LUNC-BUSD,-2.01244111
17,2023-05-10 17:36:48.425773,1.Start with 1000 BUSD and sell for 999.05015 USDT 2.Buy YFI with 0.13748806808807326 USDT 3.Sell YFI to get 997.9642915968203 BUSD. Scenario 7.BUSD-USDT/YFI-USDT/YFI-BUSD,-2.03570840
15,2023-05-10 17:36:49.078891,1.Start with 1000 BUSD and sell for 999.05015 USDT 2.Buy YFI with 0.13748806808807326 USDT 3.Sell YFI to get 997.9642915968203 BUSD. Scenario 7.BUSD-USDT/YFI-USDT/YFI-BUSD,-2.03570840
19,2023-05-10 17:36:47.462714,1.Start with 1000 BUSD and buy 0.5321868525747884 ETH 2.Sell ETH to get 0.03538249544688649 BTC 3.Sell BTC to get 997.9480941972258 BUSD. Scenario 2.ETH-BUSD/ETH-BTC/BTC-BUSD,-2.05190580
22,2023-05-10 17:37:07.529670,1.Start with 1000 BUSD and buy 0.0354554189576231 BTC 2.Buy 3.166398015319053 BNB with BTC 3.Sell BNB to get 997.9329225811056 BUSD. Scenario 1.BTC-BUSD/BNB-BTC/BNB-BUSD,-2.06707742


In [ ]:
# variant with threading, but i don't like the performance

# class Arbitrage:
#     print('Initializing...')
    
#     def __init__(self):
#         self.ta_table = get_tatable()
#         self.initial_df = pd.DataFrame(columns=['symbol', 'bidPrice', 'bidQty', 'askPrice', 'askQty'])
#         self.calculations_df = pd.DataFrame(columns=['Time', 'Descripion', 'Profit'])
#         self.count = 0
# #         self.lock = threading.Lock()
#         self.stream_on = True
        
#         ta_table_thread = threading.Thread(target=self.update_ta_table, daemon=True)
#         ta_table_thread.start()

#     def update_ta_table(self):
#         while self.stream_on:
            
# #             with self.lock:
#             self.ta_table = get_tatable()
#             print('TA table is updated')
#             time.sleep(15)

#     def message_handler(self, _, message):
#         # remove the problem with Unnamed:0
#         x = adjust_data_to_table(message).reset_index(drop=True)

#         # check if the the downloading goes every second
#         self.count += 1

#         # just checking where the process is
#         print(f'{self.count} downloaded {len(message)} units of data')

#         # creating udpdatable pandas dataframe table
#         self.initial_df = pd.concat([self.initial_df, x])

#         # remove duplicates with new data
#         self.initial_df = self.initial_df.drop_duplicates(subset='symbol', keep='last')

# #         with self.lock:
#         prepare = ta_preparation(self.ta_table, self.initial_df)
#         calculate = ta_calculation(prepare)

#         # make dataframe to save calculations to .csv file
#         self.calculations_df = pd.concat([self.calculations_df, calculate])
#         # self.calculations_df.to_csv('profits.csv', index=False)
            
#         # try feather instead of csv
#         feather.write_dataframe(self.calculations_df, 'profits.feather')
#         # how to read feather
#         # dfcf = feather.read_dataframe('profits.feather')
    
#     def trading(self):

#         my_client = SpotWebsocketStreamClient(on_message=self.message_handler)
#         my_client.ticker()
# #          the duration of datastream
#         time.sleep(40)

#         # unsubscribe
#         my_client.ticker(action=SpotWebsocketStreamClient.ACTION_UNSUBSCRIBE)

#         # the pause after datastream ended
#         time.sleep(10)
#         logging.debug("closing ws connection")
# #         print("closing ws connection")
#         my_client.stop()
#         self.stream_on = False

# bot = Arbitrage()
# bot.trading()

In [ ]:
# class Arbitrage:
#     def __init__(self):
#         self.ta_table = get_tatable()
#         self.initial_df = pd.DataFrame(columns=['symbol', 'bidPrice', 'bidQty', 'askPrice', 'askQty'])
#         self.calculations_df = pd.DataFrame(columns=['Time', 'Descripion', 'Profit'])
#         self.lock = threading.Lock()
        
#         # start a timer to update ta_table every 15 seconds
#         threading.Timer(15, self.update_ta_table).start()

#     def update_ta_table(self):
#         with self.lock:
#             self.ta_table = get_tatable()
#         print('TA table is updated')

#         # start a new timer to update ta_table in 15 seconds
#         threading.Timer(15, self.update_ta_table).start()

#     def message_handler(self, _, message):
#         # remove the problem with Unnamed:0
#         x = adjust_data_to_table(message).reset_index(drop=True)

#         # creating udpdatable pandas dataframe table
#         self.initial_df = pd.concat([self.initial_df, x])

#         # remove duplicates with new data
#         self.initial_df = self.initial_df.drop_duplicates(subset='symbol', keep='last')

#         with self.lock:
#             prepare = ta_preparation(self.ta_table, self.initial_df)
#             calculate = ta_calculation(prepare)

#             # make dataframe to save calculations to .csv file
#             self.calculations_df = pd.concat([self.calculations_df, calculate])
#             # self.calculations_df.to_csv('profits.csv', index=False)

#             # try feather instead of csv
#             feather.write_dataframe(self.calculations_df, 'profits.feather')
#             # how to read feather
#             # dfcf = feather.read_dataframe('profits.feather')

#     def trading(self):
#         my_client = SpotWebsocketStreamClient(on_message=self.message_handler)
#         my_client.ticker()
#         # the duration of datastream
#         time.sleep(30)

#         # unsubscribe
#         my_client.ticker(action=SpotWebsocketStreamClient.ACTION_UNSUBSCRIBE)

#         # the pause after datastream ended
#         time.sleep(10)
#         logging.debug("closing ws connection")
#         my_client.stop()

# bot = Arbitrage()
# bot.trading()

In [ ]:
# # using Queue
# class Arbitrage:
#     print('Initializing...')
    
#     def __init__(self):
#         self.ta_table = None
#         self.initial_df = pd.DataFrame(columns=['symbol', 'bidPrice', 'bidQty', 'askPrice', 'askQty'])
#         self.calculations_df = pd.DataFrame(columns=['Time', 'Descripion', 'Profit'])
#         self.count = 0
#         self.queue = queue.Queue()
        
#     def update_ta_table(self):
#         client = Client("", "")
#         while True:
#             ta_table = get_tatable()
#             self.queue.put(ta_table)
#             time.sleep(15)
            
#     def message_handler(self, _, message):
#         # remove the problem with Unnamed:0
#         x = adjust_data_to_table(message).reset_index(drop=True)
        
#         # check if the the downloading goes every second
#         self.count += 1
#         print(f'{self.count} downloaded {len(message)} units of data')
        
#         # make dataframe to save calculations
#         self.initial_df = pd.concat([self.initial_df, x])
#         self.initial_df = self.initial_df.drop_duplicates(subset='symbol', keep='last')
        
#         try:
#             ta_table = self.queue.get(block=False)
#             prepare = ta_preparation(ta_table, self.initial_df)
#             calculate = ta_calculation(prepare)
#             self.calculations_df = pd.concat([self.calculations_df, calculate])
#             feather.write_dataframe(self.calculations_df, 'profits.feather')
#         except queue.Empty:
#             pass

#     def trading(self):
#         ta_table_thread = threading.Thread(target=self.update_ta_table, daemon=True)
#         ta_table_thread.start()

#         my_client = SpotWebsocketStreamClient(on_message=self.message_handler)
#         my_client.ticker()
#         # the duration of datastream
#         time.sleep(30)
        
#         my_client.ticker(action=SpotWebsocketStreamClient.ACTION_UNSUBSCRIBE)
#         time.sleep(10)
#         my_client.stop()

# bot = Arbitrage()
# bot.trading()

In [ ]:
# simple variant without threading
# made class to evade making global variables
# class Arbitrage:
    
#     print('Initializing the programm')
#     ta_table = get_tatable()
    
#     def __init__(self):

#         self.initial_df = pd.DataFrame(columns=['symbol', 'bidPrice', 'bidQty', 'askPrice', 'askQty'])
#         self.calculations_df = pd.DataFrame(columns=['Time', 'Descripion', 'Profit'])
#         self.count = 0

#     def message_handler(self, _, message):
#         # remove the problem with Unnamed:0
#         x = adjust_data_to_table(message).reset_index(drop=True)

#         # check if the the downloading goes every second
#         self.count += 1

#         # just checking where the process is
#         print(f'{self.count} downloaded {len(message)} units of data')

#         # creating udpdatable pandas dataframe table
#         self.initial_df = pd.concat([self.initial_df, x])

#         # remove duplicates with new data
#         self.initial_df = self.initial_df.drop_duplicates(subset='symbol', keep='last')

#         prepare = ta_preparation(self.ta_table, self.initial_df)
#         calculate = ta_calculation(prepare)

#         # make dataframe to save calculations to .csv file
#         self.calculations_df = pd.concat([self.calculations_df, calculate])
# #         self.calculations_df.to_csv('profits.csv', index=False)
        
#         # try feather instead of csv
#         feather.write_dataframe(self.calculations_df, 'profits.feather')
#         # how to read feather
# #         dfcf = feather.read_dataframe('profits.feather')
    
#     def start_trading(self):
#         my_client = SpotWebsocketStreamClient(on_message=self.message_handler)

        
# #         time.sleep(30)
#         # subscribe to all symbols ticker stream
#         my_client.ticker()

#         # the duration of datastream
#         time.sleep(30)

#         # unsubscribe
#         my_client.ticker(action=SpotWebsocketStreamClient.ACTION_UNSUBSCRIBE)

#         # the pause after datastream ended
#         time.sleep(10)
#         logging.debug("closing ws connection")
# #         print("closing ws connection")
#         my_client.stop()
        
# bot = Arbitrage()
# bot.start_trading()